# Merging files

## Importing the libraries

In [182]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [183]:
## importing datasets
diagnosis_dict = {"ClmDiagnosisCode_1":str,"ClmDiagnosisCode_2":str,"ClmDiagnosisCode_3":str,"ClmDiagnosisCode_4":str,
             "ClmDiagnosisCode_5":str,"ClmDiagnosisCode_6":str,"ClmDiagnosisCode_7":str,"ClmDiagnosisCode_8":str,
             "ClmDiagnosisCode_9":str,"ClmDiagnosisCode_10":str}
PATH_DIR = "..\\data\\Provider fraud detection data"
df_ben_train = pd.read_csv(PATH_DIR + "\\Train_Beneficiarydata-1542865627584.csv", parse_dates = ["DOB","DOD"])
df_ben_test = pd.read_csv(PATH_DIR + "\\Test_Beneficiarydata-1542969243754.csv", parse_dates = ["DOB","DOD"])
df_in_train = pd.read_csv(PATH_DIR + "\\Train_Inpatientdata-1542865627584.csv",dtype = diagnosis_dict)
df_in_test = pd.read_csv(PATH_DIR + "\\Test_Inpatientdata-1542969243754.csv",dtype = diagnosis_dict)
df_out_train = pd.read_csv(PATH_DIR + "\\Train_Outpatientdata-1542865627584.csv",dtype = diagnosis_dict)
df_out_test = pd.read_csv(PATH_DIR + "\\Test_Outpatientdata-1542969243754.csv",dtype = diagnosis_dict)
df_icd9_icd10 = pd.read_csv("..\data\icd9_to_icd10_cm_mapping.csv",dtype = {'icd9cm': str, 'icd10cm': str})
df_icd10_hcc = pd.read_excel("..\data\PY 2024 Proposed Clinical Revision Part C Model ICD-10 Mappings.xlsx"
                             ,skiprows=3,dtype = {'2020_CMS-HCC': str, '2024_CMS-HCC': str})
df_icd10_hcc = df_icd10_hcc[:-5]

In [184]:
## appending files
df_ben = df_ben_train.append(df_ben_test[df_ben_train.columns.to_list()]).reset_index(drop = True)
df_in = df_in_train.append(df_in_test[df_in_train.columns.to_list()]).reset_index(drop = True)
df_out = df_out_train.append(df_out_test[df_out_train.columns.to_list()]).reset_index(drop = True)

C:\Users\MR321NC\AppData\Local\Temp\ipykernel_20896\2593085134.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ben = df_ben_train.append(df_ben_test[df_ben_train.columns.to_list()]).reset_index(drop = True)
C:\Users\MR321NC\AppData\Local\Temp\ipykernel_20896\2593085134.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_in = df_in_train.append(df_in_test[df_in_train.columns.to_list()]).reset_index(drop = True)
C:\Users\MR321NC\AppData\Local\Temp\ipykernel_20896\2593085134.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_out = df_out_train.append(df_out_test[df_out_train.columns.to_list()]).reset_index(drop = True)


# PATIENTS

## Adding flags to _InPatient_ and _OutPatient_

In [185]:
df_in["flag"] = "in"
df_out["flag"] = "out"

In [186]:
for col in list(set(df_in.columns) - set(df_out.columns)):
    df_out[col] = ""

In [187]:
# Reordering the columns base on InPatients' dataframe
df_out = df_out[df_in.columns]

## Appending the _InPatient_ and _OutPatient_ data

In [188]:
df_patients = df_in.append(df_out).reset_index(drop = True)

C:\Users\MR321NC\AppData\Local\Temp\ipykernel_20896\3493510314.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_patients = df_in.append(df_out).reset_index(drop = True)


In [189]:
df_patients = df_patients.sort_values(by = "BeneID").reset_index(drop = True)

# Merging the Patients Dataframe with Beneficiary

In [190]:
## merging the dataframes
df_patients_ben = pd.merge(df_patients, df_ben,how = "inner", on = "BeneID")

In [191]:
## putting all diagnosis codes and procedure codes into one column
diagnosis = ["ClmDiagnosisCode_1","ClmDiagnosisCode_2","ClmDiagnosisCode_3","ClmDiagnosisCode_4",
             "ClmDiagnosisCode_5","ClmDiagnosisCode_6","ClmDiagnosisCode_7","ClmDiagnosisCode_8",
             "ClmDiagnosisCode_9","ClmDiagnosisCode_10"]

claim_procedure = ["ClmProcedureCode_1","ClmProcedureCode_2","ClmProcedureCode_3",
                   "ClmProcedureCode_4","ClmProcedureCode_5","ClmProcedureCode_6"]

df_patients_ben["ClmDiagnosisCode"] = df_patients_ben[diagnosis].apply(lambda x: x.values.tolist(), axis=1)
df_patients_ben['ClmDiagnosisCode'] = df_patients_ben['ClmDiagnosisCode'].apply(lambda x: list(filter(lambda y: not pd.isna(y), x)))
df_patients_ben.drop(columns=diagnosis, inplace= True)

df_patients_ben["ClmProcedureCode"] = df_patients_ben[claim_procedure].apply(lambda x: x.values.tolist(), axis=1)
df_patients_ben['ClmProcedureCode'] = df_patients_ben['ClmProcedureCode'].apply(lambda x: list(filter(lambda y: not pd.isna(y), x)))
df_patients_ben.drop(columns=claim_procedure, inplace= True)

In [192]:
## replacing values 1 to 0 and 2 to 1
value_replacement_col = ['Gender', 'Race','ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']
df_patients_ben[value_replacement_col] = df_patients_ben[value_replacement_col].replace({1:0, 2:1})

In [193]:
# ## creating diagnosis codes and procedure codes
Diagnosis_Code = pd.DataFrame(list(set([a for x in df_patients_ben["ClmDiagnosisCode"] for a in x]))).rename(columns={0:"Diagnosis Code"})
# Procedure_Code = pd.DataFrame(list(set([int(a) for x in df_patients_ben["ClmProcedureCode"] for a in x]))).rename(columns={0:"Procedure Code"})

In [194]:
# ## exporting data
# df_patients_ben.to_csv(r"../data/processed_data/master_data.csv",index=False)
# Diagnosis_Code.to_csv(r"../data/processed_data/Diagnosis_Code.csv",index=False)
# Procedure_Code.to_csv(r"../data/processed_data/Procedure_Code.csv",index=False)

## mapping icd 9 to icd 10 and hcc

In [195]:
df_icd9_icd10.head()

,icd9cm,icd10cm
0,10,A000
1,11,A001
2,19,A009
3,20,A0100
4,21,A011


In [196]:
df_icd9_icd10["icd10cm"] = df_icd9_icd10["icd10cm"].str.lower()
df_icd10_hcc["Diagnosis_Code"] = df_icd10_hcc["Diagnosis_Code"].str.lower()

In [197]:
df_icd10_hcc.head(2)

,Diagnosis_Code,Description,2020_CMS-HCC,2024_CMS-HCC,2020 CMS-HCC Model Category (V24) for 2024 Payment Year,2024 Proposed CMS-HCC Model Category (V28) for 2024 Payment Year
0,a0103,Typhoid pneumonia,115,NaN,Yes,No
1,a0104,Typhoid arthritis,39,92,Yes,Yes


In [198]:
set_d1 = set(Diagnosis_Code["Diagnosis Code"])
print("count of diagnosis codes in master data icd9: " + str(len(set_d1)))
set_d2 = set(df_icd9_icd10["icd9cm"])
print("count of diagnosis codes in mapping data icd9: " + str(len(set_d2)))
print("count of overlapping diagnosis codes icd9 : " + str(len(set_d1.intersection(set_d2))))

count of diagnosis codes in master data icd9: 11227
count of diagnosis codes in mapping data icd9: 14220
count of overlapping diagnosis codes icd9 : 10585


In [199]:
set_d1 = set(df_icd10_hcc["Diagnosis_Code"])
print("count of icd10 diagnosis codes in icd10-hcc mapping data: " + str(len(set_d1)))
set_d11 = set(Diagnosis_Code["Diagnosis Code"])
set_d22 = set(df_icd9_icd10["icd9cm"])

print("count of icd10 diagnosis codes in icd9-icd10 mapping data: " + str(len(set_d11.intersection(set_d22))))
set_111 = set_d11.intersection(set_d22)
print(len(set_111.intersection(set_d1)))

count of icd10 diagnosis codes in icd10-hcc mapping data: 10006
count of icd10 diagnosis codes in icd9-icd10 mapping data: 10585
0


In [200]:
df_patients_ben_exploded = df_patients_ben.explode("ClmDiagnosisCode")

In [201]:
len(df_patients_ben_exploded)

3210596

In [202]:
df_patients_ben_exploded = df_patients_ben_exploded.merge(df_icd9_icd10, how = "inner",left_on = "ClmDiagnosisCode",right_on = "icd9cm")

In [203]:
columns = ['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode', 'flag', 'DOB',
       'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator', 'State', 'County',
       'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'ClmDiagnosisCode','icd9cm', 'icd10cm']
df_patients_ben_exploded = df_patients_ben_exploded.drop_duplicates(subset = columns)

In [204]:
df_patients_ben_exploded.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,flag,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,ClmDiagnosisCode,ClmProcedureCode,icd9cm,icd10cm
0,BENE100000,CLM126832,2009-01-08,2009-01-08,PRV57172,50,PHY383401,NaN,NaN,,NaN,0.0,,,out,1938-01-03,NaT,0,0,0,49,430,12,12,1,1,1,1,1,1,1,0,1,1,1,0,0,120,30,79678,[],79678,r85615
1,BENE100195,CLM199701,2009-02-17,2009-02-17,PRV54267,50,PHY342324,NaN,NaN,,V7612,0.0,,,out,1932-01-07,NaT,1,1,0,25,400,12,12,1,1,0,1,0,1,0,0,1,0,1,5000,1068,320,90,79678,[],79678,r85615
2,BENE100195,CLM199701,2009-02-17,2009-02-17,PRV54267,50,PHY342324,NaN,NaN,,V7612,0.0,,,out,1932-07-01,NaT,1,1,0,25,400,12,12,1,1,0,1,0,1,0,0,1,0,1,5000,1068,320,90,79678,[],79678,r85615
3,BENE100774,CLM229981,2009-03-05,2009-03-05,PRV51933,80,PHY321444,NaN,NaN,,V762,0.0,,,out,1935-01-09,NaT,1,0,0,22,170,12,12,1,0,1,1,1,1,0,0,1,1,1,0,0,790,30,79678,[],79678,r85615
4,BENE100774,CLM229981,2009-03-05,2009-03-05,PRV51933,80,PHY321444,NaN,NaN,,V762,0.0,,,out,1935-09-01,NaT,1,0,0,22,170,12,12,1,0,1,1,1,1,0,0,1,1,1,0,0,790,30,79678,[],79678,r85615


In [207]:
df_patients_ben_final = df_patients_ben_exploded.merge(df_icd10_hcc[["Diagnosis_Code","2024_CMS-HCC"]],how = "left",left_on = "icd10cm",right_on="Diagnosis_Code")

In [211]:
len(df_patients_ben_final.dropna(subset = ["2024_CMS-HCC"]))

745075

In [212]:
len(df_patients_ben_final)

4442761

In [215]:
df_patients_ben_final = df_patients_ben_final.drop(columns = ["ClmDiagnosisCode","Diagnosis_Code","ClmProcedureCode"]).dropna(subset = ["2024_CMS-HCC"])

In [222]:
# df_patients_ben_final.groupby(by="2024_CMS-HCC")["2024_CMS-HCC"].size().sort_values(ascending= False)

In [223]:
def calculate_age(row):
    if row["DOD"] is not pd.NaT:
        val = round(((row["DOD"] - row["DOB"]).days)/365, 0)
    else:
        date = pd.to_datetime("2009-12-01")
        val = round(((date - row["DOB"]).days)/365, 0)
    return val

In [224]:
parse_dates = ["DOB", "DOD"]
df_patients_ben_final[parse_dates] = df_patients_ben_final[parse_dates].apply(pd.to_datetime)
df_patients_ben_final["Age"] = df_patients_ben_final.apply(calculate_age, axis=1)

In [227]:
df_patients_ben_final["ClaimStartDt"] = pd.to_datetime(
    df_patients_ben_final['ClaimStartDt'], errors='ignore')
df_patients_ben_final["ClaimEndDt"] = pd.to_datetime(
    df_patients_ben_final['ClaimEndDt'], errors='ignore')
df_patients_ben_final["claim_duration_num"] = (
    df_patients_ben_final["ClaimEndDt"] - df_patients_ben_final["ClaimStartDt"]).dt.days

In [228]:
df_patients_ben_final

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,flag,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,icd9cm,icd10cm,2024_CMS-HCC,Age,claim_duration_num
184972,BENE100001,CLM626521,2009-10-12,2009-10-12,PRV52145,10,PHY430032,NaN,PHY418077,,42731,0.0,,,out,1939-01-08,NaT,0,0,0,33,420,12,12,0,1,1,1,1,1,1,0,0,1,0,0,0,2530,540,4271,i472,238,71.0,0
184973,BENE100001,CLM626521,2009-10-12,2009-10-12,PRV52145,10,PHY430032,NaN,PHY418077,,42731,0.0,,,out,1939-08-01,NaT,0,0,0,33,420,12,12,0,1,1,1,1,1,1,0,0,1,0,0,0,2530,540,4271,i472,238,70.0,0
184974,BENE100077,CLM719420,2009-12-08,2009-12-08,PRV55892,1000,PHY373876,NaN,PHY373876,,NaN,0.0,,,out,1931-01-10,NaT,0,0,0,39,550,12,12,1,1,1,1,1,0,0,0,0,0,1,0,1068,1330,120,4271,i472,238,79.0,0
184975,BENE100138,CLM707634,2009-11-30,2009-11-30,PRV55368,30,PHY312425,NaN,NaN,,NaN,0.0,,,out,1930-01-02,NaT,1,0,0,34,350,12,12,0,0,0,0,0,0,0,0,0,0,1,0,0,410,430,4271,i472,238,80.0,0
184976,BENE100146,CLM203371,2009-02-19,2009-02-19,PRV56347,20,PHY350070,NaN,NaN,,NaN,0.0,,,out,1941-01-10,NaT,1,0,0,43,130,12,7,0,1,1,1,1,0,0,1,0,1,1,0,0,2730,250,4271,i472,238,69.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442749,BENE99203,CLM593510,2009-09-22,2009-09-22,PRV54748,1500,PHY382084,NaN,NaN,,80600,0.0,,,out,1920-12-01,NaT,1,0,Y,10,120,12,12,0,0,0,1,0,0,0,0,1,0,1,5000,1068,1630,30,80606,s14117a,180,89.0,0
4442753,BENE99455,CLM269601,2009-03-27,2009-03-31,PRV51347,1800,PHY396637,PHY362200,NaN,,95901,0.0,,,out,1919-01-03,NaT,1,0,0,29,10,12,12,0,1,1,0,0,1,0,0,1,1,1,0,0,5450,620,80199,s06890a,399,91.0,4
4442754,BENE99455,CLM269601,2009-03-27,2009-03-31,PRV51347,1800,PHY396637,PHY362200,NaN,,95901,0.0,,,out,1919-01-03,NaT,1,0,0,29,10,12,12,0,1,1,0,0,1,0,0,1,1,1,0,0,5450,620,80199,s069x0a,399,91.0,4
4442756,BENE99455,CLM269601,2009-03-27,2009-03-31,PRV51347,1800,PHY396637,PHY362200,NaN,,95901,0.0,,,out,1919-03-01,NaT,1,0,0,29,10,12,12,0,1,1,0,0,1,0,0,1,1,1,0,0,5450,620,80199,s06890a,399,91.0,4
